In [1]:
#import re
# Affiche le répertoire courant
#current_directory = os.getcwd()
#print("Répertoire courant :", current_directory)
#import os
#os.chdir('/Users/sodagayethiam/Documents/Formations/Parcours_data_engineer/projet_jobmarket/JobMarket/processing')


Répertoire courant : /Users/sodagayethiam/Documents/Formations/Parcours_data_engineer/projet_jobmarket/JobMarket/processing


In [1]:
#import packages
import sys
sys.path.append('../')
import glob 

import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # Init tqdm for seeing progression of apply in pandas
from common.utils import translate_to_english
from common.utils import clean_text
from common.utils import extract_salary_info
from common.utils import get_infos_location, load_location_cache, save_location_cache

import pickle
from datetime import datetime
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)  # show all columns of the dataframe 

#pd.set_option('display.max_colwidth', None)  # show all lines of the dataframe 


In [2]:
#define data and cache paths
raw_path = '../outputs/raw/'
cache_path = '../outputs/intermediate/location_cache.pkl'

#read the data
jobs_adzuna_pkl = glob.glob(f'{raw_path}*jobs_adzuna*.pkl')
jobs_adzuna = pd.concat([pd.read_pickle(file) for file in jobs_adzuna_pkl], axis=0)

jobs_themuse_pkl = glob.glob(f'{raw_path}*jobs_muse*.pkl')
jobs_themuse = pd.concat([pd.read_pickle(file) for file in jobs_themuse_pkl], axis=0)

jobs_linkedin_pkl = glob.glob(f'{raw_path}*jobs_linkedin*.pkl')
jobs_linkedin = pd.concat([pd.read_pickle(file) for file in jobs_linkedin_pkl], axis=0)

jobs_wttj_pkl  = glob.glob(f'{raw_path}*jobs_wttj*.pkl')
jobs_wttj = pd.concat([pd.read_pickle(file) for file in jobs_wttj_pkl], axis=0)

print('shape jobs_adzuna:',jobs_adzuna.shape)
print('shape jobs_linkedin:',jobs_linkedin.shape)
print('shape jobs_themuse:',jobs_themuse.shape)
print('shape jobs_wttj:',jobs_wttj.shape)


shape jobs_adzuna: (389950, 24)
shape jobs_linkedin: (7270, 15)
shape jobs_themuse: (1980, 16)
shape jobs_wttj: (13994, 21)


In [3]:
print('Treatment of jobs adzuna')
jobs_adzuna_selected = jobs_adzuna[['title','description','contract_type', 'contract_time',
'company.display_name', 'created','location.display_name','category.label', 'salary_max' ,'salary_min', #'location.area',
'redirect_url','source', 'country']].rename(columns={'title': 'job_title', 'description': 'job_description',
'company.display_name': 'company_name','created': 'publication_datetime','location.display_name': 'location','category.label': 'category',
'redirect_url': 'offer_link', 'country':'country_adzuna'})
#'location.area': 'locations_company',

jobs_adzuna_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")

jobs_adzuna_selected['publication_time'] = jobs_adzuna_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
#jobs_adzuna_selected['country']='US'
jobs_adzuna_selected['industry']='not available'
#jobs_adzuna_selected['currency']='dollars'
jobs_adzuna_selected['levels']='not available'


#jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].astype(str)
#jobs_adzuna_selected['locations_company'] = jobs_adzuna_selected['locations_company'].str.replace(r'[\[\]]', '', regex=True)
jobs_adzuna_selected.head()

print( 'taille jobadzuna avant dedoubloonnage', jobs_adzuna_selected.shape)
#companies['contents'] = companies['description'].apply(clean_text)
jobs_adzuna_selected=jobs_adzuna_selected.drop_duplicates()
print( 'taille jobadzuna apres dedoubloonnage', jobs_adzuna_selected.shape)


taille jobadzuna avant dedoubloonnage (389950, 17)
taille jobadzuna apres dedoubloonnage (136944, 17)


In [4]:
#jobs_adzuna_selected.location.nunique() #16329
#jobs_adzuna_selected.location.count() 

np.int64(136944)

In [5]:

load_location_cache(cache_path)
jobs_adzuna_selected [['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name','country_code','codeIso_lvl4' ]]= jobs_adzuna_selected['location'].progress_apply(get_infos_location)



Cache chargé.


 16%|█▌        | 21747/136944 [4:00:10<69:00:51,  2.16s/it]

Error for location 'Hoppers Crossing, Wyndham Area': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Hoppers+Crossing%2C+Wyndham+Area&format=json&limit=1&accept-language=en (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x152364090>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=5)'))


 16%|█▌        | 21837/136944 [4:01:40<71:23:29,  2.23s/it] 

Error for location 'Myaree, Melville Area': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Myaree%2C+Melville+Area&format=json&limit=1&accept-language=en (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x152388d10>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=5)'))


100%|██████████| 136944/136944 [11:25:49<00:00,  3.33it/s]  


In [48]:
jobs_adzuna_selected.columns

Index(['job_title', 'job_description', 'contract_type', 'contract_time',
       'company_name', 'publication_datetime', 'location', 'category',
       'salary_max', 'salary_min', 'offer_link', 'source', 'country_adzuna',
       'publication_date', 'publication_time', 'industry', 'levels', 'country',
       'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name'],
      dtype='object')

In [50]:
#location_cache['Phoenix, eThekwini']
#correction_location('Phoenix, eThekwini')

In [51]:
jobs_adzuna_selected.head()

,job_title,job_description,contract_type,contract_time,company_name,publication_datetime,location,category,salary_max,salary_min,...,publication_time,industry,levels,country,state,city,postcode,latitude,longitude,currency_name
0,Registered Nurse,About the role: We now seek additional Nurses ...,NaN,full_time,Akari,2024-10-28T09:31:29Z,"Deckham, Gateshead",Healthcare & Nursing Jobs,38400.0,38400.0,...,09:31:29,not available,not available,United Kingdom,England,Gateshead,NE9 5LB,54.9468789,-1.5909164,Pound Sterling
1,Bar and Waiting Staff,Bar and Waiting Staff £12 per hour South Wales...,NaN,part_time,Catering Services International,2024-11-11T11:33:56Z,"Cardiff, Cardiff County",Hospitality & Catering Jobs,23040.0,23040.0,...,11:33:56,not available,not available,United Kingdom,Cymru / Wales,Cardiff,CF14 3QU,51.50316895,-3.178576542915388,Pound Sterling
2,Flexible Unqualified Nursery Practitioner,Come and join Partou UK for the next step in y...,permanent,part_time,Just Childcare,2024-11-05T09:01:16Z,"Knowle, Bristol",Teaching Jobs,22060.0,12384.0,...,09:01:16,not available,not available,United Kingdom,England,Bristol,BS4 1BH,51.4285779,-2.5779992,Pound Sterling
3,Registered Nurse,About the role: Please note to be considered f...,NaN,full_time,Akari,2024-10-25T10:29:16Z,"Jesmond, Newcastle Upon Tyne",Healthcare & Nursing Jobs,38400.0,38400.0,...,10:29:16,not available,not available,United Kingdom,England,Newcastle upon Tyne,NE2 2DX,54.9914956,-1.6057608,Pound Sterling
4,Registered Nurse,Please note to be considered for the Register...,NaN,full_time,Akari,2024-10-18T10:20:48Z,"Aycliffe, Newton Aycliffe",Healthcare & Nursing Jobs,38400.0,38400.0,...,10:20:48,not available,not available,United Kingdom,England,Newton Aycliffe,DL5 7JQ,54.613739,-1.5897097,Pound Sterling


In [33]:
#save jobs_adzuna_selected
#jobs_adzuna_selected.to_pickle('../outputs/intermediate/jobs_adzuna_selected.pkl')


In [55]:
jobs_themuse.head() 


,contents,name,type,publication_date,short_name,model_type,id,locations,categories,tags,refs.landing_page,company.id,company.short_name,company.name,levels.name,levels.short_name
0,<p><strong>Gemeinsam machen wir den Unterschie...,Freileitungsmonteur (w/m/d) Fahrleitungs- und ...,external,2023-01-24T00:42:10Z,freileitungsmonteur-wmd-fahrleitungs-und-strom...,jobs,10536487,"[Bad Mergentheim, Germany]",Computer and IT,[],https://www.themuse.com/jobs/siemens/freileitu...,11913,siemens,Siemens,Mid Level,mid
1,<p><strong>Mechanical Assembler</strong><br><b...,Mechanical Assembler - 1st Shift,external,2023-04-26T23:41:35Z,mechanical-assembler-1st-shift-0a7d96,jobs,11259124,"[Clayton, NC]",Software Engineering,[],https://www.themuse.com/jobs/siemens/mechanica...,11913,siemens,Siemens,Senior Level,senior
2,<p>Bring your heart to CVS Health. Every one o...,Pharmacy Technician,external,2024-06-06T12:25:46Z,pharmacy-technician-4dfa8b,jobs,14569255,"[Rockville, MD]",Healthcare,[],https://www.themuse.com/jobs/cvshealth/pharmac...,12096,cvshealth,CVS Health,Senior Level,senior
3,<p><strong>Who designs your future? You do.</s...,Sr. Systems Specialist - Smart Buildings / Aut...,external,2023-05-03T23:45:16Z,sr-systems-specialist-smart-buildings-automati...,jobs,11034620,"[Waterloo, IA]",Design and UX,[],https://www.themuse.com/jobs/siemens/sr-system...,11913,siemens,Siemens,Senior Level,senior
4,<p>Bring your heart to CVS Health. Every one o...,Pharmacy Technician Front End - Full Time,external,2024-06-06T12:33:58Z,pharmacy-technician-front-end-full-time-416939,jobs,14444118,"[Edison, NJ]",Healthcare,[],https://www.themuse.com/jobs/cvshealth/pharmac...,12096,cvshealth,CVS Health,Mid Level,mid


In [6]:
print('Treatment of jobs The Muse')

jobs_theMuse_selected = jobs_themuse[['name','contents','type', 'company.name',
'publication_date', 'locations','categories','refs.landing_page','levels.name']].rename(columns={'name': 'job_title', 'contents': 'job_description',
'type': 'contract_type','company.name': 'company_name','publication_date': 'publication_datetime','locations': 'location',
'categories': 'category','refs.landing_page': 'offer_link','levels.name':'levels'})

jobs_theMuse_selected['publication_date'] = pd.to_datetime(jobs_theMuse_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_theMuse_selected['publication_time'] = jobs_theMuse_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())

#jobs_theMuse_selected['country']='a travailler'
jobs_theMuse_selected['industry']='not available'
#jobs_theMuse_selected['currency']='not available'
jobs_theMuse_selected['contract_time']='not available'
jobs_theMuse_selected['salary_max']=0
jobs_theMuse_selected['salary_min']=0

jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].astype(str)
jobs_theMuse_selected['location'] = jobs_theMuse_selected['location'].str.replace(r'[\[\]]', '', regex=True)
#jobs_theMuse_selected['locations_company']=jobs_theMuse_selected['location']

jobs_theMuse_selected['job_description'] = jobs_theMuse_selected['job_description'].apply(clean_text)

jobs_theMuse_selected["source"] = "TheMuse"
jobs_theMuse_selected=jobs_theMuse_selected.drop_duplicates()
print( 'taille jobs_theMuse apres dedoubloonnage', jobs_theMuse_selected.shape)

jobs_theMuse_selected.head()


Treatment of jobs The Muse
taille jobs_theMuse apres dedoubloonnage (1980, 16)


,job_title,job_description,contract_type,company_name,publication_datetime,location,category,offer_link,levels,publication_date,publication_time,industry,contract_time,salary_max,salary_min,source
0,Freileitungsmonteur (w/m/d) Fahrleitungs- und ...,<p><strong>Gemeinsam machen wir den Unterschie...,external,Siemens,2023-01-24T00:42:10Z,"'Bad Mergentheim, Germany'",Computer and IT,https://www.themuse.com/jobs/siemens/freileitu...,Mid Level,2023-01-24,00:42:10,not available,not available,0,0,TheMuse
1,Mechanical Assembler - 1st Shift,<p><strong>Mechanical Assembler</strong><br><b...,external,Siemens,2023-04-26T23:41:35Z,"'Clayton, NC'",Software Engineering,https://www.themuse.com/jobs/siemens/mechanica...,Senior Level,2023-04-26,23:41:35,not available,not available,0,0,TheMuse
2,Pharmacy Technician,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:25:46Z,"'Rockville, MD'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,Senior Level,2024-06-06,12:25:46,not available,not available,0,0,TheMuse
3,Sr. Systems Specialist - Smart Buildings / Aut...,<p><strong>Who designs your future? You do.</s...,external,Siemens,2023-05-03T23:45:16Z,"'Waterloo, IA'",Design and UX,https://www.themuse.com/jobs/siemens/sr-system...,Senior Level,2023-05-03,23:45:16,not available,not available,0,0,TheMuse
4,Pharmacy Technician Front End - Full Time,<p>Bring your heart to CVS Health. Every one o...,external,CVS Health,2024-06-06T12:33:58Z,"'Edison, NJ'",Healthcare,https://www.themuse.com/jobs/cvshealth/pharmac...,Mid Level,2024-06-06,12:33:58,not available,not available,0,0,TheMuse


In [7]:
load_location_cache(cache_path)
jobs_theMuse_selected [['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name','country_code','codeIso_lvl4' ]]= jobs_theMuse_selected['location'].progress_apply(get_infos_location)


Cache chargé.


100%|██████████| 1980/1980 [03:31<00:00,  9.37it/s]


In [28]:

#df[['currency', 'min_salary', 'max_salary']] = df['salary'].apply(extract_salary_info)
jobs_wttj.head()

,title,company,type_company,site_company,type_contract,salary,contents,date_public,monthsOfExperience,employmentType,remote,qualifications,validThrough,logo,addressCountry,postalCode,addressLocality,streetAddress,industry,addressRegion,link_job
0,Manager Conseil en transformation data (H/F),iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3>🚀 <strong>iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,No remote work,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
1,Consultant.e senior Stratégie & Conseil,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,No remote work,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
2,Consultant.e Senior Supply chain,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,No remote work,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
3,Consultant.e Contract Management,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,No remote work,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...
4,Manager Conseil Services Financiers - Offre Ba...,iQo,Organization,https://www.iqo.eu/,Permanent contract,Salary:Not specified,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,Not specified,FULL_TIME,No remote work,,2024-11-20T13:12:43.000Z,https://cdn-images.welcometothejungle.com/C0fD...,FR,75008,Paris,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...","Organization / Management, Strategy, Change Ma...",Paris,https://www.welcometothejungle.com/en/companie...


In [29]:
print('Treatment of jobs Welcome to te jungle')

jobs_wttj_selected = jobs_wttj[['title', 'company', 'type_contract', 'contents', 'date_public',
       'employmentType','addressCountry', 'addressLocality',  'industry', 'link_job', 'salary', 'postalCode', 'streetAddress']].rename(
columns={'title': 'job_title', 'contents': 'job_description',
'employmentType': 'contract_time','company': 'company_name','date_public': 'publication_datetime','addressLocality': 'location','type_contract':'contract_type',
'addressCountry': 'country_wttj','link_job': 'offer_link','streetAddress': 'streetAdress_wttj','postalCode': 'postalCode_wttj'})



jobs_wttj_selected['publication_date'] = pd.to_datetime(jobs_wttj_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_wttj_selected['publication_time'] = jobs_wttj_selected['publication_datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").time())
jobs_wttj_selected['industry']='not available'
jobs_wttj_selected['category']='not available'

jobs_wttj_selected[['currency_wttj', 'salary_min', 'salary_max']] = jobs_wttj_selected['salary'].apply(extract_salary_info)
jobs_wttj_selected = jobs_wttj_selected.drop('salary', axis=1)

jobs_wttj_selected['location'] = jobs_wttj_selected['location'].astype(str)
jobs_wttj_selected['location'] = jobs_wttj_selected['location'].str.replace(r'[\[\]]', '', regex=True)
#jobs_wttj_selected['locations_company']=jobs_wttj_selected['location']

jobs_wttj_selected['job_description'] = jobs_wttj_selected['job_description'].apply(clean_text)
jobs_wttj_selected["levels"] = "not available"

jobs_wttj_selected["source"] = "Wttj"
jobs_wttj_selected=jobs_wttj_selected.drop_duplicates()


jobs_wttj_selected.head()

Treatment of jobs Welcome to te jungle


,job_title,company_name,contract_type,job_description,publication_datetime,contract_time,country_wttj,location,industry,offer_link,postalCode_wttj,streetAdress_wttj,publication_date,publication_time,category,currency_wttj,salary_min,salary_max,levels,source
0,Manager Conseil en transformation data (H/F),iQo,Permanent contract,<h3>🚀 <strong>iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,75008,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,Not,0.0,0.0,not available,Wttj
1,Consultant.e senior Stratégie & Conseil,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,75008,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,Not,0.0,0.0,not available,Wttj
2,Consultant.e Senior Supply chain,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,75008,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,Not,0.0,0.0,not available,Wttj
3,Consultant.e Contract Management,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,75008,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,Not,0.0,0.0,not available,Wttj
4,Manager Conseil Services Financiers - Offre Ba...,iQo,Permanent contract,<h3><strong>🚀 iQo – le conseil accélérateur de...,2024-08-22T13:12:43Z,FULL_TIME,FR,Paris,not available,https://www.welcometothejungle.com/en/companie...,75008,"91 Rue du Faubourg Saint-Honoré, 75008 Paris, ...",2024-08-22,13:12:43,not available,Not,0.0,0.0,not available,Wttj


In [30]:
#cache_path = '../outputs/intermediate/location_cache.pkl'

load_location_cache(cache_path)
jobs_wttj_selected [['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name','country_code','codeIso_lvl4' ]]= jobs_wttj_selected['streetAdress_wttj'].progress_apply(get_infos_location)

print( 'taille jobs_theMuse_selected', jobs_wttj_selected.shape)
#save_location_cache(cache_path)

Cache chargé.


100%|██████████| 13994/13994 [18:03<00:00, 12.91it/s]  

taille jobs_theMuse_selected (13994, 29)


In [13]:
#jobs_linkedin.link.unique()
jobs_linkedin.head()

,title,company,location,link,salary,salary_min,salary_max,seniority_level,employment_type,job_function,job_industry,posted_time,company_link,source,ingest_date
0,Healthcare Scheduler - Fully Remote,Vigilance Health,United States,https://www.linkedin.com/jobs/view/healthcare-...,$18.00 - $19.00,$18.00,$19.00,Entry level,Full-time,Health Care Provider,Hospitals and Health Care,2024-11-02,https://www.linkedin.com/company/vigilance-hea...,linkedin,2024-11-02 19:54:25.139908
1,Marketing Assistant,PyjamaHR,"New York, NY",https://www.linkedin.com/jobs/view/marketing-a...,,,,Entry level,Full-time,Administrative,Professional Training and Coaching,2024-11-02,https://www.linkedin.com/company/computercore?...,linkedin,2024-11-02 19:54:25.139908
2,Online Proofreader,IEM,United States,https://www.linkedin.com/jobs/view/online-proo...,,,,Not Applicable,Full-time,Sales and Business Development,"Retail Apparel and Fashion, Design Services, a...",2024-11-02,https://www.linkedin.com/company/beyond-yoga?t...,linkedin,2024-11-02 19:54:25.139908
3,Bagger,Jungle Jim's International Market,"Fairfield, OH",https://www.linkedin.com/jobs/view/bagger-at-j...,,,,Entry level,Part-time,Information Technology,Human Resources Services,2024-11-02,https://www.linkedin.com/company/timelyfind?tr...,linkedin,2024-11-02 19:54:25.139908
4,Data Entry Operator,Computer CORE (Community OutReach and Education),United States,https://www.linkedin.com/jobs/view/data-entry-...,$45.00 - $50.00,$45.00,$50.00,Entry level,Full-time,Other,Retail,2024-11-02,https://www.linkedin.com/company/jungle-jim's-...,linkedin,2024-11-02 19:54:25.139908


In [12]:
print('Treatment of jobs Linkedin')

jobs_linkedin_selected = jobs_linkedin[['title','employment_type', 'company','salary_min','salary_max',
'posted_time','location','job_industry','link','source','seniority_level','job_function']].rename(columns={'title': 'job_title', 'employment_type': 'contract_time',
'company': 'company_name','posted_time': 'publication_datetime','job_industry': 'industry','link': 'offer_link','seniority_level':'levels','job_function':'category'})

#jobs_linkedin_selected['publication_date'] = pd.to_datetime(jobs_adzuna_selected['publication_datetime']).dt.strftime("%Y-%m-%d")
jobs_linkedin_selected['job_description']='not available'
#jobs_linkedin_selected['currency']='dollars'
jobs_linkedin_selected['publication_time'] =None

jobs_linkedin_selected['publication_date'] = pd.to_datetime(jobs_linkedin_selected['publication_datetime']).dt.strftime("%Y-%m-%d")

jobs_linkedin_selected['contract_type']='not available'

#jobs_linkedin_selected['locations_company']='not available'

#jobs_linkedin_selected['job_description'] = jobs_linkedin_selected['job_description'].apply(clean_text)
jobs_linkedin_selected.head()

Treatment of jobs Linkedin


,job_title,contract_time,company_name,salary_min,salary_max,publication_datetime,location,industry,offer_link,source,levels,category,job_description,publication_time,publication_date,contract_type
0,Storyboard Artist - Unannounced Disney Junior ...,Full-time,The Walt Disney Company,"$65,000.00","$95,000.00",2024-11-09,"Glendale, CA",Entertainment Providers,https://www.linkedin.com/jobs/view/storyboard-...,linkedin,Associate,"Design, Art/Creative, and Information Technology",not available,None,2024-11-09,not available
1,Private Equity Associate,Full-time,"HRPivot, Inc",,,2024-11-09,"Chicago, IL",Internet Publishing,https://www.linkedin.com/jobs/view/private-equ...,linkedin,Entry level,Management and Manufacturing,not available,None,2024-11-09,not available
2,Photographer,Full-time,CoCreativ,,,2024-11-09,"New York, NY",Software Development,https://www.linkedin.com/jobs/view/photographe...,linkedin,Mid-Senior level,Other,not available,None,2024-11-09,not available
3,"Coordinator, Events (Hulu)",Full-time,The Walt Disney Company,,,2024-11-09,"Santa Monica, CA",Government Administration and Strategic Manage...,https://www.linkedin.com/jobs/view/coordinator...,linkedin,Not Applicable,Sales and Business Development,not available,None,2024-11-09,not available
4,Remote Sports Events Coordinator,Full-time,LC LUXE,$30.00,$35.00,2024-11-09,"Los Angeles, CA",IT Services and IT Consulting,https://www.linkedin.com/jobs/view/remote-spor...,linkedin,Mid-Senior level,Human Resources,not available,None,2024-11-09,not available


12884

In [13]:

load_location_cache(cache_path)

jobs_linkedin_selected [['country', 'state', 'city', 'postcode', 'latitude', 'longitude', 'currency_name','country_code','codeIso_lvl4' ]]= jobs_linkedin_selected['location'].progress_apply(get_infos_location)


Cache chargé.


  0%|          | 0/7270 [00:00<?, ?it/s]

100%|██████████| 7270/7270 [22:29<00:00,  5.39it/s]  


101267
0


In [31]:
print('Fusion of the 4 sources')

jobs1 = pd.concat([jobs_linkedin_selected, jobs_theMuse_selected], axis=0, ignore_index=True)
#jobs1 = jobs1.reset_index(drop=True)
jobs2 = pd.concat([jobs1, jobs_adzuna_selected], axis=0, ignore_index=True)
#jobs2 = jobs2.reset_index(drop=True)
#jobs_wttj_selected = jobs_wttj_selected.reset_index(drop=True)

jobs_final = pd.concat([jobs2, jobs_wttj_selected], axis=0, ignore_index=True)
jobs_final=jobs_final.drop_duplicates()
#jobs = pd.concat([jobs2, jobs_wttj_selected], axis=0)

shape=jobs_final.shape
print(f"shape of fusionned dataframe : {shape}")
#Taille du dataframe fusionné : (16269, 21)

Fusion of the 4 sources
shape of fusionned dataframe : (160188, 30)


In [80]:
#clean the final dataframe
#jobs_final=jobs_final.drop("country_adzuna", axis=1)
jobs_final.country.value_counts()

country
Australia                      22094
Not found                      18060
France                         17446
België / Belgique / Belgien    17325
Brasil                         13507
                               ...  
قطر                                1
الإمارات العربية المتحدة           1
Ελλάς                              1
Argentina                          1
Monaco                             1
Name: count, Length: 61, dtype: int64

In [ ]:
pd.crosstab(jobs_final.currency_name, jobs_final.country_code)
#jobs_final.source.value_counts()
#pd.crosstab(jobs_final.country, jobs_final.country_adzuna)
#pd.crosstab(jobs_final.country, jobs_final.country_wttj)


In [ ]:
#Correct country whch are not found by geopy from the location via the country in adzuna

country_dict = { 'au': 'Australia', 'be': 'België / Belgique / Belgien', 'br': 'Brasil', 'ca': 'Canada','de': 'Deutschland','es': 'España', 
                  'fr': 'France', 'gb': 'United Kingdom', 'in': 'India','mx': 'México', 'nl': 'Nederland', 'nz': 'New Zealand / Aotearoa',
                  'pl': 'Polska', 'sg': 'Singapore', 'us': 'United States','za': 'South Africa'}


jobs_final = jobs_final.reset_index(drop=True)
jobs_final.loc[jobs_final['country'] == 'Not found', 'country'] = jobs_final.apply( lambda row: country_dict.get(row['country_adzuna'], 'Unknown'), axis=1)
jobs_final=jobs_final.drop("country_adzuna", axis=1)

#jobs_final.country.value_counts()

In [137]:
#jobs_final['contents'] = jobs_final['contents'].progress_apply(translate_to_english)


#jobs_final.currency_name.value_counts()
#jobs_final[jobs_final.country=='Not found']

In [95]:
jobs_final = jobs_final[jobs_final['job_title'].notnull()] #make sure we have'nt jobs without title

#standardise contract_time
jobs_final['contract_time'] = jobs_final['contract_time'].fillna('not available')
jobs_final['contract_time'] = jobs_final['contract_time'].str.lower().str.replace('-', '_').str.strip()
jobs_final.loc[jobs_final['contract_time'] == "internship", 'contract_time'] = 'intern'
jobs_final.loc[jobs_final['contract_time'] == "contract", 'contract_time'] = 'contractor'

#standardise company_name
jobs_final['company_name'] = jobs_final['company_name'].fillna('not available')
jobs_final['company_name'] = jobs_final['company_name'].str.upper().str.strip()

#replace the value unknow in industry
jobs_final.loc[jobs_final['industry'] == "Unknown", 'industry'] = 'not available'

#clean salary and replace the unknown values by 0
jobs_final['salary_min'] = pd.to_numeric(jobs_final['salary_min'], errors='coerce').fillna(0)
jobs_final['salary_max'] = pd.to_numeric(jobs_final['salary_max'], errors='coerce').fillna(0)

#format to datetime
jobs_final['publication_date'] = pd.to_datetime(jobs_final['publication_date'], errors='coerce')
jobs_final['publication_time'] = pd.to_datetime(jobs_final['publication_time'],  format='%H:%M:%S',errors='coerce')
jobs_final['publication_time'] = jobs_final['publication_time'].apply(lambda x: x.time() if pd.notna(x) else pd.NaT)
jobs_final['publication_time'] = jobs_final['publication_time'].fillna(pd.to_datetime('00:00:00').time())

#replace the industry by category where we haven't values 
jobs_final = jobs_final.reset_index(drop=True)
jobs_final.loc[jobs_final['industry'] == "not available", 'industry'] = jobs_final['category']

jobs_final['levels'] = jobs_final['levels'].str.lower().str.strip()

jobs_final['job_description'] = jobs_final['job_description'].fillna('not available')

jobs_final['contract_type'] = jobs_final['contract_type'].fillna('not available')
jobs_final['contract_type'] = jobs_final['contract_type'].str.lower().str.strip()
jobs_final.loc[jobs_final['contract_type'] == "permanent contract", 'contract_type'] = 'permanent'

#replace not found coordonate
jobs_final['latitude'] = jobs_final['latitude'].replace('Not found', '0').astype(float)
jobs_final['longitude'] = jobs_final['longitude'].replace('Not found', '0').astype(float)

#replace Unknown values by not found to standardize
jobs_final.loc[jobs_final['state'] == "Unknown", 'state'] = 'Not found'
jobs_final.loc[jobs_final['city'] == "Unknown", 'city'] = 'Not found'
jobs_final.loc[jobs_final['postcode'] == "Unknown", 'postcode'] = 'Not found'

#replace specific values of countries not found by the script
jobs_final.loc[jobs_final['country'] == "Australia", 'country_code'] = 'au'
jobs_final.loc[jobs_final['country'] == "United Kingdom", 'country_code'] = 'gb'
jobs_final.loc[jobs_final['country'] == "België / Belgique / Belgien", 'country_code'] = 'be'

#replace unknowns currencies 
currency_dict = {  'be': 'Euro', 'fr': 'Euro','br': 'BRL',  'au': 'Australian Dollar',  'es': 'Euro', 
    'pl': 'Polish Zloty',  'it': 'Euro', 'de': 'Euro', 'nl': 'Euro',  'gb': 'Pound Sterling', 'cf': 'CFA Franc',  'at': 'Euro',  'ci': 'CFA Franc', 'pt': 'Euro',  # Portugal
    'tr': 'Turkish Lira', 'sk': 'EUR',  'rs': 'Serbian Dinar',  'ie': 'Euro',  # Ireland
    'ro': 'Romanian Leu',  'bg': 'Bulgarian Lev',  'tw': 'New Taiwan Dollar', 'ee': 'EUR',   'lt': 'EUR',  'gr': 'EUR', 'mc': 'EUR' }
jobs_final = jobs_final.reset_index(drop=True)
jobs_final.loc[jobs_final['currency_name'].isin(['Unknown', 'Not found']), 'currency_name'] = jobs_final.apply( lambda row: currency_dict.get(row['country_code'], 'Unknown'), axis=1)

#Clean the html balises
jobs_final['job_description'] =jobs_final['job_description'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
jobs_final['job_description'] = jobs_final['job_description'].apply(clean_text)

#translate text from french or german to english
jobs_final['job_title'] = jobs_final['job_title'].progress_apply(translate_to_english)
jobs_final['job_description'] = jobs_final['job_description'].progress_apply(translate_to_english)

#delete unnecessary columns
jobs_final=jobs_final.drop("postalCode_wttj", axis=1)
jobs_final=jobs_final.drop("streetAdress_wttj", axis=1)
jobs_final=jobs_final.drop("currency_wttj", axis=1)
jobs_final=jobs_final.drop("country_adzuna", axis=1)
jobs_final=jobs_final.drop("country_wttj", axis=1)

#jobs_final=jobs_final.drop("Unknown", axis=1)


In [8]:

#'country_adzuna', 'country_wttj

In [10]:
#jobs_final['job_description'] = jobs_final['job_description'].apply(clean_text)

#jobs_final['job_title'] = jobs_final['job_title'].progress_apply(translate_to_english)
from functools import lru_cache
@lru_cache(maxsize=None)
def cached_translate(text):
    return translate_to_english(text)


jobs_final['job_description'] = jobs_final['job_description'].progress_apply(cached_translate)
#jobs_final['job_description'] = jobs_final['job_description'].progress_apply(translate_to_english)


  0%|          | 0/160188 [00:00<?, ?it/s]

 33%|███▎      | 52125/160188 [02:51<05:56, 303.47it/s]   


KeyboardInterrupt: 

In [16]:
from langdetect import detect

# addjobs_final['lang'] = jobs_final['job_description'].progress_apply(detect)
jobs_final['lang'] = jobs_final['job_description'].progress_apply(detect)



100%|██████████| 160188/160188 [09:58<00:00, 267.84it/s]


In [17]:
jobs_final['lang'].value_counts()


lang
en       106250
nl        14518
pt        12809
es        10808
fr         7439
it         4514
pl         3366
ca          139
cs          128
lv          102
de           28
af           19
sl           13
uk           10
no            6
zh-cn         6
ru            5
cy            4
ko            3
ro            3
sv            3
ja            2
sk            2
so            2
tl            2
da            1
th            1
vi            1
hu            1
lt            1
sw            1
et            1
Name: count, dtype: int64

In [18]:

# remove the nuntranslated 
non_english_jobs = jobs_final[jobs_final['lang'] != 'en']
english_jobs = jobs_final[jobs_final['lang'] == 'en']


In [20]:
non_english_jobs.loc[:, 'job_description'] = non_english_jobs['job_description'].progress_apply(cached_translate)


  0%|          | 0/53938 [00:00<?, ?it/s]

 76%|███████▌  | 40883/53938 [59:18<18:56, 11.49it/s]    


KeyboardInterrupt: 

In [ ]:
non_english_jobs['lang'] = non_english_jobs['job_description'].progress_apply(detect)
non_english_jobs['lang'].value_counts()


In [ ]:
#jobs_final = pd.concat([english_jobs, non_english_jobs], ignore_index=True)


In [ ]:
#jobs_final = jobs_final[jobs_final['lang'] == 'en']

# delete the temporary column
#jobs_final=jobs_final.drop("lang", axis=1)


In [134]:
#pd.crosstab(jobs_final.country, jobs_final.country_code)


In [ ]:
town_data=pd.read_pickle('../outputs/intermediate/town_data.pkl')
town_data.country_code.value_counts(dropna=False)
#calculate the indicator of the country by taking the average of the cities to fill the NA values 
indicator_avg_by_country = town_data.groupby('country_code')[[
    'Quality_of_Life_Index', 'Purchasing_Power_Index', 'Safety_Index', 'Health_Care_Index',
    'Cost_of_Living_Index', 'Rent_Index', 'Groceries_Index', 'Crime_Index', 
    'Climate_Index', 'Pollution_Index', 'Traffic_Index', 'Affordability_Index'
]].mean()

city_country = pd.merge(town_data, indicator_avg_by_country, on='country_code', how='left', suffixes=('', '_country_avg'))
#mzrge jobs with informations about city
jobs = pd.merge(jobs_final,  town_data[['Quality_of_Life_Index', 'Purchasing_Power_Index',
       'Safety_Index', 'Health_Care_Index', 'Cost_of_Living_Index',
       'Rent_Index', 'Groceries_Index', 'Crime_Index', 'Climate_Index',
       'Pollution_Index', 'Traffic_Index', 'Affordability_Index',
       'city']], on='city', how='left')
#merge jobs with informations about countries
jobs_city_country = pd.merge(jobs, indicator_avg_by_country, on='country_code', how='left', suffixes=('', '_country_avg'))
#replace the nan values for cities with values of the countries  

index_life_col = ['Quality_of_Life_Index', 'Purchasing_Power_Index', 'Safety_Index', 'Health_Care_Index', 'Cost_of_Living_Index',
       'Rent_Index', 'Groceries_Index', 'Crime_Index', 'Climate_Index', 'Pollution_Index', 'Traffic_Index', 'Affordability_Index']

for column in index_life_col:
    jobs_city_country[column] = jobs_city_country[column].fillna(jobs_city_country[f'{column}_country_avg'])

# delete avg indicators by country in final dataframe
jobs_city_country.drop(columns=[f'{column}_country_avg' for column in index_life_col], inplace=True)
#replace the last NA values by the mean
jobs_city_country[index_life_col] = jobs_city_country[index_life_col].fillna(jobs_city_country[index_life_col].mean())


In [ ]:
#pd.crosstab(jobs_final.currency_name, jobs_final.country)


In [171]:

#jupyter nbconvert --to script process_jobs.ipynb

In [131]:
jobs_final.loc[jobs_final['country_code'].isin( ['Not found']), 'country'].value_counts()
jobs_final.currency_name.value_counts()


currency_name
Euro                  57388
Australian Dollar     33330
BRL                   13507
US Dollar             13116
Unknown                6250
Pound Sterling         5409
Polish Zloty           4876
Mexican Peso           4859
Singapore Dollar       4614
Indian Rupee           4381
New Zealand Dollar     4333
Canadian Dollar        3983
Rand                   3316
CFA Franc               314
Czech Koruna            134
Philippine Peso          60
Uganda Shilling          48
Colombian Peso           37
Yuan Renminbi            36
Norwegian Krone          34
Swiss Franc              19
Malaysian Ringgit        17
EUR                      17
Turkish Lira             16
Tunisian Dinar            9
Egyptian Pound            8
Serbian Dinar             8
New Israeli Sheqel        7
Sol                       6
Yen                       6
Won                       5
Rupiah                    5
Moroccan Dirham           5
Dong                      5
Danish Krone              4
Saudi 

In [232]:
#Avoid  double entries in the dataframe

jobs_final.loc[:, 'clef_doublons'] = jobs_final['job_title'].astype(str) + '-' + jobs_final['company_name'].astype(str) + '-' + jobs_final['city'].astype(str)+'-'+ jobs_final['levels'].astype(str)+'-'+ jobs_final['job_description'].astype(str)+'-'+ jobs_final['contract_type'].astype(str)
doublons_freq = jobs_final['clef_doublons'].value_counts()
print(doublons_freq[doublons_freq > 1])  # Afficher uniquement les doublons

aggregation_rules = {col: 'first' for col in jobs_final.columns}
aggregation_rules.update({
    'salary_min': 'max',  
    'salary_max': 'max',  
    'publication_datetime': 'max',  
    'publication_date': 'max',  
    'publication_time': 'max',  
    'publication_date': 'min',    
})
jobs_final = jobs_final.groupby('clef_doublons').agg(aggregation_rules).reset_index(drop=True)
jobs_final = jobs_final.drop('clef_doublons', axis=1)

jobs_final.shape



In [ ]:
jobs_final.to_pickle('../outputs/final/jobs.pkl')
#jobs_final=pd.read_pickle('../outputs/final/jobs.pkl')
#'''FINNNNNNNNNNNNNNNNNNNNN'''

In [ ]:
jobs_final.dtypes
#is_datetime = pd.api.types.is_datetime64_any_dtype(jobs_final['publication_date'])
#print(is_datetime) ``
jobs_final